In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture



## Preprocessing

In [2]:
data_df = pd.read_csv("dataset_exercise_5_clustering_highway_traffic.csv",sep=";")

data_df

,PORTAL,Date,time_from,time_to,Interval_5,SPEED_MS_AVG,flow
0,"E4S 56,780",20210101,00:00:00,00:05:00,0,18.56,39
1,"E4S 56,780",20210101,00:05:00,00:10:00,1,20.39,18
2,"E4S 56,780",20210101,00:10:00,00:15:00,2,19.27,26
3,"E4S 56,780",20210101,00:15:00,00:20:00,3,19.52,52
4,"E4S 56,780",20210101,00:20:00,00:25:00,4,20.52,52
...,...,...,...,...,...,...,...
104838,"E4S 56,780",20211231,23:35:00,23:40:00,283,19.58,115
104839,"E4S 56,780",20211231,23:40:00,23:45:00,284,19.47,87
104840,"E4S 56,780",20211231,23:45:00,23:50:00,285,19.77,130
104841,"E4S 56,780",20211231,23:50:00,23:55:00,286,18.79,129


In [3]:
# Sort the DataFrame 'data_df' by columns "Date" and "Interval_5"
data_df.sort_values(["Date", "Interval_5"])

# Extract unique dates from the sorted DataFrame
days = np.unique(data_df[['Date']].values.ravel())
# Calculate the total number of unique days
ndays = len(days)

# Group the DataFrame 'data_df' by the "Date" column
day_subsets_df = data_df.groupby(["Date"])

# Define the total number of 5-minute intervals in a day
nintvals = 288

# Create a matrix 'vectorized_day_dataset' filled with NaN values
vectorized_day_dataset = np.zeros((ndays, nintvals))
vectorized_day_dataset.fill(np.nan)

# Loop through each unique day
for i in range(0, ndays):
    # Get the DataFrame corresponding to the current day
    df_t = day_subsets_df.get_group(days[i])

    # Loop through each row in the current day's DataFrame
    for j in range(len(df_t)):
        # Get the current day's DataFrame
        df_t = day_subsets_df.get_group(days[i])

        # Extract the "Interval_5" and "flow" values and populate 'vectorized_day_dataset'
        vectorized_day_dataset[i, df_t.iloc[j]["Interval_5"]] = df_t.iloc[j]["flow"]

# Print the resulting 'vectorized_day_dataset'
print(vectorized_day_dataset)

/var/folders/ww/hnj0z7l577b2_jzdg6515pfm0000gn/T/ipykernel_2960/2050002240.py:22: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  df_t = day_subsets_df.get_group(days[i])
/var/folders/ww/hnj0z7l577b2_jzdg6515pfm0000gn/T/ipykernel_2960/2050002240.py:27: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  df_t = day_subsets_df.get_group(days[i])


[[ 39.  18.  26. ...  32.  39.  34.]
 [ 30.  32.  27. ...  44.  41.  39.]
 [ 36.  44.  52. ...  50.  45.  23.]
 ...
 [ 20.  34.  31. ...  38.  42.  36.]
 [ 36.  40.  25. ...  38.  56.  35.]
 [ 33.  32.  34. ... 130. 129. 117.]]


In [5]:
nans_per_day = np.sum(np.isnan(vectorized_day_dataset),1)
print('number of days with missing value',np.size(np.where(nans_per_day > 0),1))

number of days with missing value 28


In [6]:
n_clusters = 10
clusters = None
#print(np.where(nans_per_day > 0)[0])
vectorized_day_dataset_no_nans = vectorized_day_dataset[np.where(nans_per_day == 0)[0],:]
days_not_nans = days[np.where(nans_per_day == 0)[0]]

X_train, X_val = train_test_split(vectorized_day_dataset_no_nans, test_size=0.2, random_state=42)

In [6]:
nans_per_day = np.sum(np.isnan(vectorized_day_dataset),1)
print('number of days with missing value',np.size(np.where(nans_per_day > 0),1))

number of days with missing value 28


## Clustering

In [14]:
# ============================
# Advanced-track grid search
# ============================
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture


X0 = vectorized_day_dataset  
mask = ~np.any(np.isnan(X0), axis=1)
X = X0[mask]

# 1) Split train/validation
X_train, X_val = train_test_split(X, test_size=0.2, random_state=42)

def safe_internal_metrics(X_fit, labels):
    labels = np.asarray(labels)
    valid = labels != -1
    uniq = np.unique(labels[valid])
    if uniq.size < 2:
        return dict(sil=np.nan, ch=np.nan, db=np.nan, n_clusters=uniq.size, frac_noise=np.mean(labels==-1))
    try:
        sil = silhouette_score(X_fit[valid], labels[valid])
        ch  = calinski_harabasz_score(X_fit[valid], labels[valid])
        db  = davies_bouldin_score(X_fit[valid], labels[valid])
        return dict(sil=sil, ch=ch, db=db, n_clusters=uniq.size, frac_noise=np.mean(labels==-1))
    except Exception:
        return dict(sil=np.nan, ch=np.nan, db=np.nan, n_clusters=uniq.size, frac_noise=np.mean(labels==-1))

def centroids_from_train(X_fit, labels):
    cents = []
    ids = []
    for c in sorted([c for c in np.unique(labels) if c != -1]):
        cents.append(X_fit[labels == c].mean(axis=0))
        ids.append(c)
    if not cents:
        return np.empty((0, X_fit.shape[1])), np.array([])
    return np.vstack(cents), np.array(ids)

def assign_to_nearest_centroid(X_new, centroids):
    if centroids.shape[0] == 0:
        return np.full(X_new.shape[0], -1)
    x2 = np.sum(X_new**2, axis=1, keepdims=True)
    c2 = np.sum(centroids**2, axis=1, keepdims=True).T
    cross = X_new @ centroids.T
    d2 = x2 + c2 - 2*cross
    return np.argmin(d2, axis=1)

def external_errors(X_val, cents, val_assign):
    if cents.shape[0] == 0 or np.all(val_assign == -1):
        return np.nan, np.nan
    preds = cents[val_assign]
    mae  = mean_absolute_error(X_val, preds)
    rmse = np.sqrt(mean_squared_error(X_val, preds))
    return mae, rmse


results = []

# 2) KMEANS
for k in [3,4,5,6,7,8,10,12]:
    km = KMeans(n_clusters=k, n_init="auto", random_state=0)
    labels_tr = km.fit_predict(X_train)
    im = safe_internal_metrics(X_train, labels_tr)
    cents, _ = centroids_from_train(X_train, labels_tr)
    val_assign = assign_to_nearest_centroid(X_val, cents)
    mae, rmse = external_errors(X_val, cents, val_assign)
    results.append({"algo":"kmeans","params":f"k={k}","sil":im["sil"],"ch":im["ch"],"db":im["db"],
                    "n_clusters":im["n_clusters"],"noise":im["frac_noise"],"MAE":mae,"RMSE":rmse})

# 3) AGGLO
for link in ["ward","average","complete"]:
    for k in [3,4,5,6,7,8,10]:
        if link != "ward":
            ag = AgglomerativeClustering(n_clusters=k, linkage=link)
        else:
            ag = AgglomerativeClustering(n_clusters=k, linkage="ward")
        labels_tr = ag.fit_predict(X_train)
        im = safe_internal_metrics(X_train, labels_tr)
        cents, _ = centroids_from_train(X_train, labels_tr)
        val_assign = assign_to_nearest_centroid(X_val, cents)
        mae, rmse = external_errors(X_val, cents, val_assign)
        results.append({"algo":"agglo","params":f"k={k},link={link}","sil":im["sil"],"ch":im["ch"],"db":im["db"],
                        "n_clusters":im["n_clusters"],"noise":im["frac_noise"],"MAE":mae,"RMSE":rmse})

# 4) GMM
for k in [3,4,5,6,7,8,10]:
    for cov in ["full","diag","tied","spherical"]:
        gm = GaussianMixture(n_components=k, covariance_type=cov, random_state=0)
        gm.fit(X_train)
        labels_tr = gm.predict(X_train)
        im = safe_internal_metrics(X_train, labels_tr)
        cents, _ = centroids_from_train(X_train, labels_tr)
        val_assign = assign_to_nearest_centroid(X_val, cents)
        mae, rmse = external_errors(X_val, cents, val_assign)
        results.append({"algo":"gmm","params":f"k={k},cov={cov}","sil":im["sil"],"ch":im["ch"],"db":im["db"],
                        "n_clusters":im["n_clusters"],"noise":im["frac_noise"],"MAE":mae,"RMSE":rmse})

# 5) DBSCAN  (tip: scale X first if needed; eps is data-scale sensitive)

for eps in [2.0, 3.0, 4.0, 5.0]:     
    for ms in [3,5,10]:
        db = DBSCAN(eps=eps, min_samples=ms)
        labels_tr = db.fit_predict(X_train)
        im = safe_internal_metrics(X_train, labels_tr)
        cents, _ = centroids_from_train(X_train, labels_tr)
        val_assign = assign_to_nearest_centroid(X_val, cents)
        mae, rmse = external_errors(X_val, cents, val_assign)
        results.append({"algo":"dbscan","params":f"eps={eps},min_samples={ms}","sil":im["sil"],"ch":im["ch"],"db":im["db"],
                        "n_clusters":im["n_clusters"],"noise":im["frac_noise"],"MAE":mae,"RMSE":rmse})

df = pd.DataFrame(results)


df_rank = df.copy()


df_rank["MAE_filled"] = df_rank["MAE"].fillna(df_rank["MAE"].max() + 1e6)


df_rank["sil_filled"] = df_rank["sil"].fillna(-1e6)


df_rank["db_filled"] = df_rank["db"].fillna(df_rank["db"].max() + 1e6)

# Now rank properly
df_rank["rank_ext"] = df_rank["MAE_filled"].rank(method="min")        
df_rank["rank_sil"] = (-df_rank["sil_filled"]).rank(method="min")      
df_rank["rank_db"]  = df_rank["db_filled"].rank(method="min")        

# Sum ranks
df_rank["rank_sum"] = df_rank[["rank_ext","rank_sil","rank_db"]].sum(axis=1)

# Sort
df_sorted = df_rank.sort_values("rank_sum").reset_index(drop=True)

df_sorted.head(20)


,algo,params,sil,ch,db,n_clusters,noise,MAE,RMSE,MAE_filled,sil_filled,db_filled,rank_ext,rank_sil,rank_db,rank_sum
0,agglo,"k=7,link=average",0.279675,54.003427,0.845864,7,0.0,28.667871,43.153036,28.667871,0.279675,0.845864,41.0,6.0,8.0,55.0
1,agglo,"k=4,link=complete",0.279928,92.209611,1.007734,4,0.0,28.709749,43.538289,28.709749,0.279928,1.007734,43.0,5.0,9.0,57.0
2,agglo,"k=3,link=complete",0.318667,54.845992,0.721327,3,0.0,33.555019,49.292986,33.555019,0.318667,0.721327,56.0,1.0,4.0,61.0
3,agglo,"k=10,link=average",0.241411,39.605748,0.802769,10,0.0,28.234210,42.384047,28.234210,0.241411,0.802769,33.0,23.0,6.0,62.0
4,agglo,"k=7,link=complete",0.274291,54.475372,1.116768,7,0.0,28.759843,43.268379,28.759843,0.274291,1.116768,44.0,7.0,11.0,62.0
5,agglo,"k=3,link=average",0.281186,4.878789,0.519491,3,0.0,42.941645,61.760881,42.941645,0.281186,0.519491,57.0,4.0,1.0,62.0
6,agglo,"k=5,link=complete",0.283361,76.685367,1.234156,5,0.0,28.775626,43.528410,28.775626,0.283361,1.234156,46.0,3.0,14.0,63.0
7,agglo,"k=8,link=complete",0.254478,48.866330,1.096967,8,0.0,28.498034,42.679963,28.498034,0.254478,1.096967,34.0,21.0,10.0,65.0
8,agglo,"k=3,link=ward",0.285200,132.731090,1.264086,3,0.0,31.536967,50.406958,31.536967,0.285200,1.264086,49.0,2.0,15.0,66.0
9,agglo,"k=6,link=complete",0.274044,63.667614,1.209533,6,0.0,28.765797,43.272943,28.765797,0.274044,1.209533,45.0,8.0,13.0,66.0


## Evaluation

In [26]:
data_df_eval = pd.read_csv("evaluation_dataset_exercise_5_clustering_highway_traffic.csv",sep=";")

In [27]:
def vectorize_dataset(df, date_col="Date", interval_col="Interval_5", value_col="flow", n_intervals=288):
    df_sorted = df.sort_values([date_col, interval_col])
    out = (df_sorted.pivot(index=date_col, columns=interval_col, values=value_col)
                    .reindex(columns=range(n_intervals))
                    .sort_index())
    X = out.to_numpy()
    days = out.index.to_numpy()
    return X, days

# Apply to evaluation dataset
vectorized_day_dataset_eval, eval_days = vectorize_dataset(
    data_df_eval, date_col="Date", interval_col="Interval_5", value_col="flow"
)

print("Evaluation dataset vectorized:", vectorized_day_dataset_eval.shape)

Evaluation dataset vectorized: (80, 288)


In [28]:
def centroids_from_labels(X, labels):
    cents = [X[labels==c].mean(axis=0) for c in sorted(set(labels) - {-1})]
    return np.vstack(cents) if cents else np.empty((0, X.shape[1]))

def assign_to_nearest_centroid(X_new, C):
    if C.shape[0] == 0:
        return np.full(X_new.shape[0], -1)
    x2 = np.sum(X_new**2, axis=1, keepdims=True)
    c2 = np.sum(C**2, axis=1, keepdims=True).T
    return np.argmin(x2 + c2 - 2*(X_new @ C.T), axis=1)

def eval_on_dataset(X_train_like, X_eval_like, model):
    labels = model.fit_predict(X_train_like) if hasattr(model, "fit_predict") else model.predict(X_train_like)
    C = centroids_from_labels(X_train_like, labels)
    idx = assign_to_nearest_centroid(X_eval_like, C)
    if C.shape[0] == 0 or np.all(idx == -1):
        return np.nan, np.nan
    preds = C[idx]
    mae = mean_absolute_error(X_eval_like, preds)
    rmse = np.sqrt(mean_squared_error(X_eval_like, preds))
    return mae, rmse

# 3. Final Model

best_model = AgglomerativeClustering(n_clusters=7, linkage="average")


# 4. Run final evaluation


X_full, _ = vectorize_dataset(data_df)  
X_full = X_full[~np.any(np.isnan(X_full), axis=1)]
X_eval = vectorized_day_dataset_eval[~np.any(np.isnan(vectorized_day_dataset_eval), axis=1)]

mae, rmse = eval_on_dataset(X_full, X_eval, best_model)

print("Final evaluation on evaluation dataset:")
print("MAE:", mae)
print("RMSE:", rmse)

Final evaluation on evaluation dataset:
MAE: 21.144787423826216
RMSE: 30.947059458287207
